In [68]:
!pip install turicreate

In [69]:
!pip freeze

absl-py==0.10.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
asgiref==3.3.1
astor==0.8.1
astropy==4.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.6
atomicwrites==1.4.0
attrs==20.3.0
audioread==2.1.9
autograd==1.3
Babel==2.9.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.1.1
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cachetools==4.2.1
catalogue==1.0.0
certifi==2020.12.5
cffi==1.14.4
chainer==7.4.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.0
coremltools==3.3
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.5
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.21
daft==0.0.4
dask==2.12.0
dataclasses==0.8
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.3
distributed==1.25.3
Django==3.1.6
dlib==19.18.0
dm-t

In [70]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

import sys
sys.path.append("..")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
#customers = pd.read_csv('../data/recommend_1.csv') 
#transactions = pd.read_csv('../data/trx_data.csv')
import pandas as pd
#customers = pd.read_csv("/content/per_user_distance_purchase_count.csv", encoding= 'unicode_escape')
data = pd.read_csv("/content/ratings.csv", encoding= 'unicode_escape')
data.head()

,user_id,product_number,summation_distance_Purchasecount
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [72]:
#float(customers.user_id)

In [73]:
'''data = pd.melt(customers.set_index('user_id')['product_number'].apply(pd.Series).reset_index(), 
             id_vars=['user_id'],
             value_name='product_number') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['user_id', 'product_number']) \
    .agg({'product_number': 'count'}) \
    .rename(columns={'product_number': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'product_number': 'productId'})
data['product_number'] = data['product_number'].astype(np.int64)
'''


"data = pd.melt(customers.set_index('user_id')['product_number'].apply(pd.Series).reset_index(), \n             id_vars=['user_id'],\n             value_name='product_number')     .dropna().drop(['variable'], axis=1)     .groupby(['user_id', 'product_number'])     .agg({'product_number': 'count'})     .rename(columns={'product_number': 'purchase_count'})     .reset_index()     .rename(columns={'product_number': 'productId'})\ndata['product_number'] = data['product_number'].astype(np.int64)\n"

In [74]:
data.dtypes

user_id                               int64
product_number                        int64
summation_distance_Purchasecount    float64
dtype: object

In [75]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [76]:
df_matrix = pd.pivot_table(data, values='summation_distance_Purchasecount', index='user_id', columns='product_number')

In [77]:
df_matrix 

product_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.5,NaN,4.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,4.5,NaN,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,2.0,NaN,3.5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,3.5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [79]:
df_matrix_norm.dtypes

product_number
1         float64
2         float64
3         float64
4         float64
5         float64
           ...   
193581    float64
193583    float64
193585    float64
193587    float64
193609    float64
Length: 9724, dtype: object

In [80]:
#df_matrix_norm = df_matrix_norm.astype('category')

In [81]:
df_matrix_norm

product_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.777778,NaN,0.777778,NaN,NaN,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.777778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750,0.777778,NaN,0.555556,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.444444,NaN,NaN,NaN,NaN,NaN,0.375,NaN,NaN,NaN,0.375,NaN,NaN,NaN,0.625,NaN,0.777778,0.666667,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,0.833333,NaN,NaN,0.750,NaN,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,0.777778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500,0.777778,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,0.444444,0.333333,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,0.777778,NaN,NaN,NaN,NaN,NaN,0.875,NaN,NaN,0.25,NaN,0.625,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,0.555556,0.625,0.625,NaN,NaN,0.555556,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# create a table for input to the modeling  
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['user_id'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm

(96716, 3)


,user_id,product_number,scaled_purchase_freq
0,1,1,0.777778
4,5,1,0.777778
6,7,1,0.888889
14,15,1,0.444444
16,17,1,0.888889
...,...,...,...
5913999,50,188301,0.000000
5914329,380,188301,1.000000
5914545,596,188301,1.000000
5918403,184,189333,1.000000


In [83]:
'''def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
    '''

"def normalize_data(data):\n    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')\n    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())\n    d = df_matrix_norm.reset_index()\n    d.index.names = ['scaled_purchase_freq']\n    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()\n    "

In [84]:
data=data.dropna()
data['user_id']=data['user_id'].astype(np.int64)
data['product_number']=data['product_number'].astype(np.int64)
data_dummy=data_dummy.dropna()
data_dummy['user_id']=data_dummy['user_id'].astype(np.int64)
data_dummy['product_number']=data_dummy['product_number'].astype(np.int64)
data_norm=data_norm.dropna()
data_norm['user_id']=data_norm['user_id'].astype(np.int64)
data_norm['product_number']=data_norm['product_number'].astype(np.int64)

In [85]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [86]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [87]:
# constant variables to define field names include:
user_id = 'user_id'
item_id = 'product_number'
users_to_recommend = list(data[user_id])
n_rec = 10 # number of items to recommend
n_display = 10 # to display the first few rows in an output dataset

In [88]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [89]:
data = data.astype('category')
data["user_id"] = data["user_id"].astype("category")

In [90]:
data.dtypes

user_id                             category
product_number                      category
summation_distance_Purchasecount    category
dtype: object

In [91]:
#data=data.dropna()

In [92]:
#data['user_id']=data['user_id'].astype(np.int64)

In [93]:
data.dtypes

user_id                             category
product_number                      category
summation_distance_Purchasecount    category
dtype: object

In [94]:
name = 'popularity'
target = 'summation_distance_Purchasecount'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 80668 observations with 610 users and 8970 items.

Data prepared in: 0.104467s

80668 observations to process; with 8970 unique items.

recommendations finished on 1000/100836 queries. users per second: 12008.8

...

recommendations finished on 97000/100836 queries. users per second: 10416

recommendations finished on 98000/100836 queries. users per second: 10420.9

recommendations finished on 99000/100836 queries. users per second: 10422.6

recommendations finished on 100000/100836 queries. users per second: 10323.7

+---------+----------------+-------+------+
| user_id | product_number | score | rank |
+---------+----------------+-------+------+
|    1    |      3473      |  5.0  |  1   |
|    1    |      2196      |  5.0  |  2   |
|    1    |     146684     |  5.0  |  3   |
|    1    |     136355     |  5.0  |  4   |
|    1    |     97866      |  5.0  |  5   |
|    1    |      3531      |  5.0  |  6   |
|    1    |     31522      |  5.0  |  7   |
|    1    |     170597     |  5.0  |  8   |
|    1    |     50851      |  5.0  |  9   |
|    1    |     27751      |  5.0  |  10  |
+---------+----------------+-------+------+
[1008360 rows x 4 columns]



In [95]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns summation_distance_Purchasecount;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 80668 observations with 610 users and 9001 items.

Data prepared in: 0.111134s

...

recommendations finished on 97000/100836 queries. users per second: 9326.5

recommendations finished on 98000/100836 queries. users per second: 9334.04

recommendations finished on 99000/100836 queries. users per second: 9337.19

recommendations finished on 100000/100836 queries. users per second: 9340.98

+---------+----------------+-------+------+
| user_id | product_number | score | rank |
+---------+----------------+-------+------+
|    1    |      2067      |  1.0  |  1   |
|    1    |     39715      |  1.0  |  2   |
|    1    |      1203      |  1.0  |  3   |
|    1    |     92259      |  1.0  |  4   |
|    1    |      292       |  1.0  |  5   |
|    1    |      1963      |  1.0  |  6   |
|    1    |     112852     |  1.0  |  7   |
|    1    |     145150     |  1.0  |  8   |
|    1    |      912       |  1.0  |  9   |
|    1    |      3246      |  1.0  |  10  |
+---------+----------------+-------+------+
[1008360 rows x 4 columns]



In [96]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 77372 observations with 610 users and 5924 items.

Data prepared in: 0.09572s

77372 observations to process; with 5924 unique items.

recommendations finished on 1000/100836 queries. users per second: 17889.4

...

recommendations finished on 97000/100836 queries. users per second: 13525.2

recommendations finished on 98000/100836 queries. users per second: 13514.8

recommendations finished on 99000/100836 queries. users per second: 13526.2

recommendations finished on 100000/100836 queries. users per second: 13542.6

+---------+----------------+-------+------+
| user_id | product_number | score | rank |
+---------+----------------+-------+------+
|    1    |      1105      |  1.0  |  1   |
|    1    |     115122     |  1.0  |  2   |
|    1    |      6970      |  1.0  |  3   |
|    1    |     64249      |  1.0  |  4   |
|    1    |      3728      |  1.0  |  5   |
|    1    |      4141      |  1.0  |  6   |
|    1    |     86548      |  1.0  |  7   |
|    1    |     117109     |  1.0  |  8   |
|    1    |      8012      |  1.0  |  9   |
|    1    |      697       |  1.0  |  10  |
+---------+----------------+-------+------+
[1008360 rows x 4 columns]



In [97]:
#train_data_norm.groupby(by=item_id)['distance_Productcount'].mean().sort_values(ascending=False).head(20)


In [98]:
name = 'cosine'
target = 'summation_distance_Purchasecount'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 80668 observations with 610 users and 8970 items.

Data prepared in: 0.112392s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 97000/100836 queries. users per second: 2889.38

recommendations finished on 98000/100836 queries. users per second: 2874.11

recommendations finished on 99000/100836 queries. users per second: 2857.76

recommendations finished on 100000/100836 queries. users per second: 2827.1

+---------+----------------+---------------------+------+
| user_id | product_number |        score        | rank |
+---------+----------------+---------------------+------+
|    1    |      1240      |  0.5919824658878265 |  1   |
|    1    |      1089      |  0.5426407166258999 |  2   |
|    1    |      1968      |  0.4644849615753012 |  3   |
|    1    |      2918      | 0.46148567855673495 |  4   |
|    1    |      1036      |  0.4393117102996382 |  5   |
|    1    |      1291      |  0.4039863909363116 |  6   |
|    1    |      1200      |  0.4005932606086529 |  7   |
|    1    |      2762      |  0.3768132839884077 |  8   |
|    1    |      1527      |  0.3644522841645296 |  9   |
|    1    |      1270      |  0.3613960660954632 |  10  |
+---------+----------------+---------------------+------+
[1008360 rows x 4 columns]



In [99]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns summation_distance_Purchasecount;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 80668 observations with 610 users and 9001 items.

Data prepared in: 0.1069s

...

recommendations finished on 97000/100836 queries. users per second: 2736.47

recommendations finished on 98000/100836 queries. users per second: 2723.44

recommendations finished on 99000/100836 queries. users per second: 2709.9

recommendations finished on 100000/100836 queries. users per second: 2690.43

+---------+----------------+---------------------+------+
| user_id | product_number |        score        | rank |
+---------+----------------+---------------------+------+
|    1    |      1196      | 0.11845514568358816 |  1   |
|    1    |      2115      | 0.10912407131095207 |  2   |
|    1    |      1968      | 0.10595061485679987 |  3   |
|    1    |      2918      | 0.10582190658409558 |  4   |
|    1    |      1265      | 0.10472694117361338 |  5   |
|    1    |      1377      | 0.10453885200760127 |  6   |
|    1    |      2683      | 0.09684532631130119 |  7   |
|    1    |      1391      | 0.09054090714579477 |  8   |
|    1    |      1610      | 0.08966903723971383 |  9   |
|    1    |      924       |   0.08533421330427  |  10  |
+---------+----------------+---------------------+------+
[1008360 rows x 4 columns]



In [100]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 77372 observations with 610 users and 5924 items.

Data prepared in: 0.091979s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 97000/100836 queries. users per second: 3302.11

recommendations finished on 98000/100836 queries. users per second: 3284.97

recommendations finished on 99000/100836 queries. users per second: 3267.01

recommendations finished on 100000/100836 queries. users per second: 3240.17

+---------+----------------+----------------------+------+
| user_id | product_number |        score         | rank |
+---------+----------------+----------------------+------+
|    1    |      1580      | 0.10815036392466908  |  1   |
|    1    |      1089      | 0.09216510357066272  |  2   |
|    1    |      1291      | 0.08683964626036864  |  3   |
|    1    |      1682      |  0.079601658856805   |  4   |
|    1    |      1196      |  0.0795139523750958  |  5   |
|    1    |      1036      |  0.0758779313475053  |  6   |
|    1    |      1527      | 0.07452901671914493  |  7   |
|    1    |      2762      | 0.07225314117370442  |  8   |
|    1    |      2918      | 0.07213117732083733  |  9   |
|    1    |      1265      | 0.059398401867259636 |  10  |
+---------+----------------+----------------------+------+
[1008360 rows x 4 columns]



In [101]:
name = 'pearson'
target = 'summation_distance_Purchasecount'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 80668 observations with 610 users and 8970 items.

Data prepared in: 0.103352s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 97000/100836 queries. users per second: 2153.31

recommendations finished on 98000/100836 queries. users per second: 2145.2

recommendations finished on 99000/100836 queries. users per second: 2135.28

recommendations finished on 100000/100836 queries. users per second: 2115.15

+---------+----------------+-------+------+
| user_id | product_number | score | rank |
+---------+----------------+-------+------+
|    1    |      3473      |  5.0  |  1   |
|    1    |      2196      |  5.0  |  2   |
|    1    |     146684     |  5.0  |  3   |
|    1    |     136355     |  5.0  |  4   |
|    1    |     97866      |  5.0  |  5   |
|    1    |      3531      |  5.0  |  6   |
|    1    |     31522      |  5.0  |  7   |
|    1    |     170597     |  5.0  |  8   |
|    1    |     50851      |  5.0  |  9   |
|    1    |     27751      |  5.0  |  10  |
+---------+----------------+-------+------+
[1008360 rows x 4 columns]



In [102]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns summation_distance_Purchasecount;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 80668 observations with 610 users and 9001 items.

Data prepared in: 0.129998s

...

recommendations finished on 97000/100836 queries. users per second: 5805.12

recommendations finished on 98000/100836 queries. users per second: 5802.79

recommendations finished on 99000/100836 queries. users per second: 5801.87

recommendations finished on 100000/100836 queries. users per second: 5801.2

+---------+----------------+-------+------+
| user_id | product_number | score | rank |
+---------+----------------+-------+------+
|    1    |      2067      |  0.0  |  1   |
|    1    |     39715      |  0.0  |  2   |
|    1    |      1203      |  0.0  |  3   |
|    1    |     92259      |  0.0  |  4   |
|    1    |      292       |  0.0  |  5   |
|    1    |      1963      |  0.0  |  6   |
|    1    |     112852     |  0.0  |  7   |
|    1    |     145150     |  0.0  |  8   |
|    1    |      912       |  0.0  |  9   |
|    1    |      3246      |  0.0  |  10  |
+---------+----------------+-------+------+
[1008360 rows x 4 columns]



In [103]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 77372 observations with 610 users and 5924 items.

Data prepared in: 0.109083s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 97000/100836 queries. users per second: 2776.06

recommendations finished on 98000/100836 queries. users per second: 2756.89

recommendations finished on 99000/100836 queries. users per second: 2736.21

recommendations finished on 100000/100836 queries. users per second: 2706.87

+---------+----------------+-------+------+
| user_id | product_number | score | rank |
+---------+----------------+-------+------+
|    1    |      1105      |  1.0  |  1   |
|    1    |     115122     |  1.0  |  2   |
|    1    |      6970      |  1.0  |  3   |
|    1    |     64249      |  1.0  |  4   |
|    1    |      3728      |  1.0  |  5   |
|    1    |      4141      |  1.0  |  6   |
|    1    |     86548      |  1.0  |  7   |
|    1    |     117109     |  1.0  |  8   |
|    1    |      8012      |  1.0  |  9   |
|    1    |      697       |  1.0  |  10  |
+---------+----------------+-------+------+
[1008360 rows x 4 columns]



In [104]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on  ratings', 'Cosine Similarity on ratings', 'Pearson Similarity on ratings']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled ratings', 'Cosine Similarity on Scaled ratings', 'Pearson Similarity on Scaled ratings']

In [105]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on  ratings

Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    |          0.0           |          0.0           |
|   2    |          0.0           |          0.0           |
|   3    |          0.0           |          0.0           |
|   4    |          0.0           |          0.0           |
|   5    |          0.0           |          0.0           |
|   6    |          0.0           |          0.0           |
|   7    |          0.0           |          0.0           |
|   8    | 0.00020525451559934323 | 0.0002736726874657909  |
|   9    | 0.0001824484583105272  | 0.0002736726874657909  |
|   10   | 0.0001642036124794745  | 0.00027367268746579087 |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9

In [106]:
'''metrics = models_w_norm.evaluate(test_data_norm)
print("Accuracy: ", metrics["accuracy"])
print("Precision: ", metrics["precision"])
print("Recall: ", metrics["recall"])
'''

'metrics = models_w_norm.evaluate(test_data_norm)\nprint("Accuracy: ", metrics["accuracy"])\nprint("Precision: ", metrics["precision"])\nprint("Recall: ", metrics["recall"])\n'

In [107]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='scaled_purchase_freq', similarity_type='pearson')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 96716 observations with 610 users and 5980 items.

Data prepared in: 0.119691s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 97000/100836 queries. users per second: 2349.96

recommendations finished on 98000/100836 queries. users per second: 2333.12

recommendations finished on 99000/100836 queries. users per second: 2313.36

recommendations finished on 100000/100836 queries. users per second: 2285.12

+---------+----------------+--------------------+------+
| user_id | product_number |       score        | rank |
+---------+----------------+--------------------+------+
|    1    |      489       | 0.8643112846513983 |  1   |
|    1    |      318       | 0.8580062866161888 |  2   |
|    1    |      360       | 0.848631918689479  |  3   |
|    1    |     107141     | 0.8438651725001955 |  4   |
|    1    |      3915      | 0.8394236996553947 |  5   |
|    1    |      949       | 0.8374290377333544 |  6   |
|    1    |      6639      | 0.8367346938775511 |  7   |
|    1    |      4754      | 0.8333333333333334 |  8   |
|    1    |      2548      | 0.8333333333333334 |  9   |
|    1    |      912       | 0.8315482767135052 |  10  |
+---------+----------------+--------------------+------+
[1008360 rows x 4 columns]



In [108]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(1008360, 4)


,user_id,product_number,score,rank
0,1,489,0.864311,1
1,1,318,0.858006,2
2,1,360,0.848632,3
3,1,107141,0.843865,4
4,1,3915,0.839424,5


In [109]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedmovies'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['user_id', 'recommendedmovies']].drop_duplicates() \
        .sort_values('user_id').set_index('user_id')
    if print_csv:
        df_output.to_csv('/content/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [110]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/100836 queries. users per second: 6387.21

recommendations finished on 2000/100836 queries. users per second: 5822.25

recommendations finished on 3000/100836 queries. users per second: 6326.43

recommendations finished on 4000/100836 queries. users per second: 5884.34

recommendations finished on 5000/100836 queries. users per second: 5946.15

...

recommendations finished on 98000/100836 queries. users per second: 2731.57

recommendations finished on 99000/100836 queries. users per second: 2710.4

recommendations finished on 100000/100836 queries. users per second: 2682.3

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(610, 1)


,recommendedmovies
user_id,
1,1105|115122|6970|64249|3728|4141|86548|117109|...
2,1105|115122|6970|64249|3728|4141|86548|117109|...
3,1105|115122|6970|64249|3728|4141|86548|117109|...
4,1105|115122|6970|64249|3728|4141|86548|117109|...
5,1105|115122|6970|64249|3728|4141|86548|117109|...


In [111]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]